In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

df = pd.read_csv("https://github.com/bgweber/Twitch/raw/master/Recommendations/games-expand.csv")
x = df.drop(['label'], axis=1)
y = df['label']

model = LogisticRegression()
model.fit(x, y)

LogisticRegression()

In [4]:
import pickle
pickle.dump(model, open("logit.pkl", 'wb'))

model = pickle.load(open("logit.pkl", 'rb'))
model.predict_proba(x)

array([[0.93124982, 0.06875018],
       [0.86491573, 0.13508427],
       [0.85766917, 0.14233083],
       ...,
       [0.89969047, 0.10030953],
       [0.85766917, 0.14233083],
       [0.89969047, 0.10030953]])

In [6]:
!pip3 install mlflow
import mlflow
import mlflow.sklearn
#import shutil

model_path = "models/logit_games_v1"
#shutil.rmtree(model_path)
mlflow.sklearn.save_model(model, model_path)

loaded = mlflow.sklearn.load_model(model_path)
loaded.predict_proba(x)

  Using cached mlflow-1.26.1-py3-none-any.whl (17.8 MB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
     |████████████████████████████████| 407 kB 25.0 MB/s eta 0:00:01
  Using cached alembic-1.8.0-py3-none-any.whl (209 kB)
  Using cached databricks_cli-0.16.6-py3-none-any.whl
  Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
  Using cached sqlparse-0.4.2-py3-none-any.whl (42 kB)
  Using cached docker-5.0.3-py2.py3-none-any.whl (146 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached SQLAlchemy-1.4.37-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached importlib_metadata-4.11.4-py3-none-any.whl (18 kB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached prometheus_flask_exporter-0.20.1-py3-none-any.whl (18 kB)
     |████████████████████████████████| 63 kB 3.0 MB/s  eta 0:00:01
  Using cached

array([[0.93124982, 0.06875018],
       [0.86491573, 0.13508427],
       [0.85766917, 0.14233083],
       ...,
       [0.89969047, 0.10030953],
       [0.85766917, 0.14233083],
       [0.89969047, 0.10030953]])

In [7]:
!pip3 install tensorflow
!pip3 install keras

     |████████████████████████████████| 511.7 MB 1.7 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 77 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 45.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 59.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 2.4 MB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 90 kB/s s eta 0:00:01
     |████████████████████████████████| 1.1 MB 56.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 59.5 MB/s eta 0:00:01
     |█████████████████████████████

In [8]:
import tensorflow as tf
import keras
from keras import models, layers

# define the network structure 
model = models.Sequential()
model.add(layers.Dense(64,activation='relu',input_shape=(10,)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(
                  tf.local_variables_initializer())
    return auc
    
model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy', metrics=[auc])
history = model.fit(x, y, epochs=100, batch_size=100, 
                 validation_split = .2, verbose=0)

2022-06-01 16:05:13.713848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 16:05:13.713912: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [9]:
from keras.models import load_model
model.save("games.h5")

model = load_model('games.h5', custom_objects={'auc': auc})
model.evaluate(x, y, verbose = 0)

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
import mlflow.keras

model_path = "models/keras_games_v1"
mlflow.keras.save_model(model, model_path)

loaded = mlflow.keras.load_model(model_path, custom_objects={'auc': auc})
loaded.evaluate(x, y, verbose = 0)